In [2]:
import scipy
import numpy as np
import pandas as pd
import plotly.plotly as py
# import visplots
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from sklearn import preprocessing, metrics
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats.distributions import randint
import plotly.graph_objs as go
import plotly.tools as tls
tls.set_credentials_file(username='cap7carps', api_key='d28TqBXnTSA80L2xctmI')
from numpy import arange,array,ones
from scipy import stats

In [21]:
weatherFrame = pd.read_pickle(r'C:\GIT\weatherproj\Weather\weather_data.pki')

In [22]:
weatherFrame.shape

(18357, 10)

In [23]:
weatherFrame.head()

,avg_rain_prob,avg_temp_frc,city,date_time_period,forecast_period,avg_temp_obs,avg_ws_obs,w_observed,avg_temp_dif,rain_accuracy
0,9.0,7.0,glasgow,2017-01-03 06:00:00,-1.0,7.3,9.5,"[8.0, 8.0]",0.3,0.91
1,9.0,6.0,glasgow,2017-01-03 06:00:00,2.0,7.3,9.5,"[8.0, 8.0]",1.3,0.91
2,8.0,6.0,glasgow,2017-01-03 06:00:00,5.0,7.3,9.5,"[8.0, 8.0]",1.3,0.92
3,10.0,6.0,glasgow,2017-01-03 06:00:00,8.0,7.3,9.5,"[8.0, 8.0]",1.3,0.90
4,9.0,6.0,glasgow,2017-01-03 06:00:00,11.0,7.3,9.5,"[8.0, 8.0]",1.3,0.91


In [6]:
london = weatherFrame.ix[weatherFrame['city'] == 'london', ['forecast_period', 'avg_temp_dif', 'rain_accuracy']].groupby('forecast_period').mean()

In [7]:
glasgow = weatherFrame.ix[weatherFrame['city'] == 'glasgow', ['forecast_period', 'avg_temp_dif', 'rain_accuracy']].groupby('forecast_period').mean()
glasgownp = np.array(glasgow)

In [8]:
londonnp = np.array(london)
london.head()

,avg_temp_dif,rain_accuracy
forecast_period,,
-5.0,1.041975,0.679444
-4.0,1.006475,0.808397
-3.0,0.970789,0.790640
-2.0,0.830000,0.560000
-1.0,0.891319,0.812311


In [9]:
timeperiods = list(london.index)

In [10]:
londonnp

array([[ 1.04197531,  0.67944444],
       [ 1.00647452,  0.80839724],
       [ 0.97078853,  0.79063978],
       [ 0.83      ,  0.56      ],
       [ 0.89131944,  0.81231146],
       [ 0.96407973,  0.81982443],
       [ 0.85185185,  0.57041667],
       [ 1.15361111,  0.83088021],
       [ 1.03235793,  0.79565649],
       [ 1.00185185,  0.56583333],
       [ 1.05531097,  0.83251782],
       [ 1.10525869,  0.79911832],
       [ 0.59230769,  0.66538462],
       [ 1.13504542,  0.83449686],
       [ 1.09033079,  0.80116031],
       [ 0.82222222,  0.63038462],
       [ 1.16250874,  0.81959644],
       [ 1.06569126,  0.81994402],
       [ 0.86825397,  0.74642857],
       [ 1.17393431,  0.81063103],
       [ 1.04173028,  0.8129173 ],
       [ 0.98888889,  0.74464286],
       [ 1.1444095 ,  0.81882914],
       [ 1.14677693,  0.80055725],
       [ 1.26190476,  0.83678571],
       [ 1.1851153 ,  0.81525786],
       [ 1.18324852,  0.81223155],
       [ 1.11825397,  0.82035714],
       [ 1.21921733,

In [11]:
londonnp[:,0]

array([ 1.04197531,  1.00647452,  0.97078853,  0.83      ,  0.89131944,
        0.96407973,  0.85185185,  1.15361111,  1.03235793,  1.00185185,
        1.05531097,  1.10525869,  0.59230769,  1.13504542,  1.09033079,
        0.82222222,  1.16250874,  1.06569126,  0.86825397,  1.17393431,
        1.04173028,  0.98888889,  1.1444095 ,  1.14677693,  1.26190476,
        1.1851153 ,  1.18324852,  1.11825397,  1.21921733,  1.09276094,
        1.26587302,  1.20405311,  1.10660774,  1.27539683,  1.13357792,
        1.185581  ,  1.30793651,  1.16771488,  1.1719697 ,  1.74126984,
        1.23539483,  1.1405303 ,  1.61428571,  1.22855134,  1.14924242,
        1.32539683,  1.16848358,  1.20089059,  1.31587302,  1.22393431,
        1.12717949,  1.21984127,  1.23357792,  1.08290598,  1.07777778,
        1.19109015,  1.20371795,  1.37777778,  1.10121528,  1.28083549,
        1.31666667,  1.19170139,  1.16550388,  1.04206349,  1.2271875 ,
        1.2118863 ,  1.25952381,  1.19989583,  1.2765625 ,  1.03

In [24]:
line1 = go.Scatter(
    y = londonnp[:,0],
    x = list(london.index),
    mode = 'lines',
    name = 'London'
)


line2 = go.Scatter(
    y = glasgownp[:,0],
    x = list(london.index),
    mode = 'lines',
    name = 'Glasgow'
)

layout = go.Layout(
    title='Temp Difference, Hours before',
    yaxis=dict(title='Difference'),
    xaxis=dict(title='Hours')
)

data = [line1, line2]
fig = go.Figure(data=data, layout=layout)
# Plot and embed in ipython notebook!
py.iplot(fig)

In [25]:
line1 = go.Scatter(
    y = londonnp[:,1],
    x = list(london.index),
    mode = 'lines',
    name = 'London'
)


line2 = go.Scatter(
    y = glasgownp[:,1],
    x = list(london.index),
    mode = 'lines',
    name = 'Glasgow'
)

layout = go.Layout(
    title='Rain Accuray, Hours before',
    yaxis=dict(title='Difference'),
    xaxis=dict(title='Hours')
)

data = [line1, line2]
fig = go.Figure(data=data, layout=layout)
# Plot and embed in ipython notebook!
py.iplot(fig)